<a href="https://colab.research.google.com/github/bt4755/513/blob/main/513Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings("ignore")
!pip install edward
!pip install torchbnn
!pip install tensorflow

Mounted at /content/drive
     |████████████████████████████████| 63 kB 2.1 MB/s 
  Created wheel for edward: filename=edward-1.3.5-py3-none-any.whl size=90385 sha256=722f862825fb548d0b0c8dba75e8476e5729595a8586e77186de946ae59d1543
  Stored in directory: /root/.cache/pip/wheels/c0/d0/6f/298c809cc8db6573b8642d1473c668868a67b9ad297c2e8b65
Successfully built edward
     |████████████████████████████████| 462 kB 27.6 MB/s 


In [11]:
%load_ext tensorboard
%rm -rf "/content/drive/My Drive/513/src/logs"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
%cd "/content/drive/My Drive/513/src/"
import os
!CUDA_LAUNCH_BLOCKING=1
os.system('CUDA_LAUNCH_BLOCKING=1')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
from torch.utils.data import DataLoader
import torch
import torchvision
from torch import cuda
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch._C import NoneType
import torchbnn as bnn
import os
import sys
import numpy as np
import timeit
import matplotlib.pyplot as plt
import matplotlib
import math
import argparse
import torch
from torchvision.transforms.transforms import Resize, Compose, ToPILImage, ToTensor, Normalize
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
from torch.autograd import Variable
from torch.utils.data import Dataset
from skimage import io



/content/drive/My Drive/513/src


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
class Discriminator(nn.Module):
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            # input: N x channels_img x 64 x 64
            nn.Conv2d(
                channels_img, features_d, kernel_size=4, stride=2, padding=1
            ),
            nn.LeakyReLU(0.2),
            # _block(in_channels, out_channels, kernel_size, stride, padding)
            self._block(features_d, features_d * 2, 4, 2, 1),
            self._block(features_d * 2, features_d * 4, 4, 2, 1),
            self._block(features_d * 4, features_d * 8, 4, 2, 1),
            # After all _block img output is 4x4 (Conv2d below makes into 1x1)
            nn.Conv2d(features_d * 8, 1, kernel_size=4, stride=2, padding=0),
            #nn.Sigmoid(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )

    def forward(self, x):
        return self.disc(x)


class Generator(nn.Module):
    def __init__(self, channels_noise, channels_img, features_g):
        super(Generator, self).__init__()
        self.net = nn.Sequential(
            # Input: N x channels_noise x 1 x 1
            self._block(channels_noise, features_g * 16, 4, 1, 0),  # img: 4x4
            self._block(features_g * 16, features_g * 8, 4, 2, 1),  # img: 8x8
            self._block(features_g * 8, features_g * 4, 4, 2, 1),  # img: 16x16
            self._block(features_g * 4, features_g * 2, 4, 2, 1),  # img: 32x32
            nn.ConvTranspose2d(
                features_g * 2, channels_img, kernel_size=4, stride=2, padding=1
            ),
            # Output: N x channels_img x 64 x 64
            nn.Tanh(),
        )

    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,
            ),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
        )

    def forward(self, x):
        return self.net(x)


def initialize_weights(model):
    # Initializes weights according to the DCGAN paper
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

class pokemonDataset(Dataset):
  def __init__(self, root_dir, size, transform):
      self.root_dir = root_dir
      self.transform = transform
      self.size = size

  def __getitem__(self, index):
      img = io.imread("{root}/{idx}.png".format(root= self.root_dir, idx = index + 1))
      img = self.transform(img)
      return img, 0

  def __len__(self):
      return self.size

In [ ]:
# Hyperparameters etc.
D_SET = 'MNIST'
#D_SET ='POKE'
LEARNING_RATE = 3e-4  # could also use two lrs, one for gen and one for disc
BATCH_SIZE = 128
IMG_SIZE = 64
IMG_CHANNELS = 0
if D_SET == 'MNIST':
  IMG_CHANNELS = 1
elif D_SET == 'POKE':
  IMG_CHANNELS = 4
else:
  IMG_CHANNELS = 3
LEARNING_RATE = 2e-4
NOISE_DIM = 100
NUM_EPOCHS = 20
feature_size = 16
DATA_PATH = None
if D_SET == 'MNIST':
  DATA_PATH = "../data"
elif D_SET == 'POKE':
  DATA_PATH = "Datasets/pokemon/pokemon/"
else:
  DATA_PATH = None

  
torch.cuda.manual_seed(1)

transformer = None
dataset = None
if D_SET == 'MNIST':
  transformer = Compose(
    [
      Resize(IMG_SIZE),
      ToTensor(),
      Normalize(
        [0.5 for _ in range(IMG_CHANNELS)], [0.5 for _ in range(IMG_CHANNELS)]
      )
    ]
  )

  dataset = datasets.MNIST(root='../data', train=True, download=True, transform=transformer)
elif D_SET == 'POKE':
  transformer = Compose(
    [
      ToPILImage(),
      Resize(IMG_SIZE),
      ToTensor(),
      Normalize(
        [0.5 for _ in range(IMG_CHANNELS)], [0.5 for _ in range(IMG_CHANNELS)]
      )
    ]
  )
  dataset = pokemonDataset(DATA_PATH,721, transformer)
else:
  pass


# comment mnist above and uncomment below if train on CelebA
#dataset = datasets.ImageFolder(root="celeb_dataset", transform=transforms)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
gen = Generator(NOISE_DIM, IMG_CHANNELS, feature_size).to(device)
disc = Discriminator(IMG_CHANNELS, feature_size).to(device)
initialize_weights(gen)
initialize_weights(disc)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
#criterion = nn.BCELoss()
criterion = nn.BCEWithLogitsLoss()

fixed_noise = torch.randn((BATCH_SIZE, NOISE_DIM, 1,1)).to(device)
writer_real = SummaryWriter('logs/{}/Real'.format(D_SET))
writer_fake = SummaryWriter('logs/{}/Fake'.format(D_SET))
step = 0

gen.train()
disc.train()

for epoch in range(NUM_EPOCHS):
    # Target labels not needed! <3 unsupervised
    for batch_idx, (real, _) in enumerate(dataloader):
        real = real.to(device)
        noise = torch.randn(BATCH_SIZE, NOISE_DIM, 1, 1).to(device)
        fake = gen(noise)

        ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
        disc_real = disc(real).reshape(-1)
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real))
        disc_fake = disc(fake.detach()).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
        loss_disc = (loss_disc_real + loss_disc_fake) / 2
        disc.zero_grad()
        loss_disc.backward()
        opt_disc.step()

        ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
        output = disc(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        gen.zero_grad()
        loss_gen.backward()
        opt_gen.step()

        # Print losses occasionally and print to tensorboard
        if batch_idx % 100 == 0:
            print(
                f"Epoch [{epoch + 1}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )

            with torch.no_grad():
                fake = gen(fixed_noise)
                # take out (up to) 32 examples
                img_grid_real = torchvision.utils.make_grid(
                    real[:32], normalize=True
                )
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:32], normalize=True
                )

                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)

            step += 1

Epoch [1/20] Batch 0/469                   Loss D: 0.6938, loss G: 0.7155
Epoch [1/20] Batch 100/469                   Loss D: 0.1141, loss G: 2.1690
Epoch [1/20] Batch 200/469                   Loss D: 0.0270, loss G: 3.5435
Epoch [1/20] Batch 300/469                   Loss D: 0.4503, loss G: 1.2282
Epoch [1/20] Batch 400/469                   Loss D: 0.6299, loss G: 0.8744
Epoch [2/20] Batch 0/469                   Loss D: 0.8492, loss G: 0.2591
Epoch [2/20] Batch 100/469                   Loss D: 0.6200, loss G: 0.7189
Epoch [2/20] Batch 200/469                   Loss D: 0.6170, loss G: 0.9306
Epoch [2/20] Batch 300/469                   Loss D: 0.6372, loss G: 0.8651
Epoch [2/20] Batch 400/469                   Loss D: 0.6534, loss G: 0.7642
Epoch [3/20] Batch 0/469                   Loss D: 0.6567, loss G: 0.6581
Epoch [3/20] Batch 100/469                   Loss D: 0.6730, loss G: 0.9352
Epoch [3/20] Batch 200/469                   Loss D: 0.6562, loss G: 0.7387
Epoch [3/20] Batch

In [ ]:
%tensorboard --logdir logs

In [ ]:
#from google.colab import files
#files.upload()